In [13]:
# Step 1: Import necessary libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_table(seat_code):
  url = 'https://results.eci.gov.in/ResultAcGenFeb2025/ConstituencywiseU05'+str(seat_code)+'.htm'
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://results.eci.gov.in"
    }
  # Step 1: Fetch the webpage content
  response = requests.get(url, headers=headers)
  webpage = response.content
  # Step 2: Parse the HTML
  soup = BeautifulSoup(webpage, 'html.parser')
  # Step 3: Find the specific table (adjust the class name or other attributes as necessary)
  table = soup.find('table', {'class': 'table table-striped table-bordered'})
  # Step 4: Define the headers
  headers = ['S.N.', 'Candidate', 'Party', 'EVM Votes', 'Postal Votes', 'Total Votes', '% of Votes']
  # Step 5: Extract the rows
  rows = []
  for tr in table.find_all('tr')[1:-1]:  # Skip the header row and last total row
      cells = tr.find_all('td')
      row = [cell.get_text().strip() for cell in cells]
      rows.append(row)
  # Step 6: Create a DataFrame and return it
  data = pd.DataFrame(rows, columns=headers)
  data['Constituency'] = seat_code
  return data

def check_valid(seat_code):
  url = 'https://results.eci.gov.in/ResultAcGenFeb2025/ConstituencywiseU05'+str(seat_code)+'.htm'
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://results.eci.gov.in"
    }
  # Step 1: Fetch the webpage content
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
    return True
  else:
    return False

# A

In [17]:
# Extracting election result data for all seats
table = pd.DataFrame(columns=['S.N.', 'Candidate', 'Party', 'EVM Votes', 'Postal Votes', 'Total Votes', '% of Votes', 'Constituency'])
for seat in range(1,71):
  table = pd.concat([table, get_table(seat)], ignore_index=True)
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   S.N.          769 non-null    object
 1   Candidate     769 non-null    object
 2   Party         769 non-null    object
 3   EVM Votes     769 non-null    object
 4   Postal Votes  769 non-null    object
 5   Total Votes   769 non-null    object
 6   % of Votes    769 non-null    object
 7   Constituency  769 non-null    object
dtypes: object(8)
memory usage: 48.2+ KB


In [18]:
table['S.N.'] = table['S.N.'].astype(int)
table['EVM Votes'] = table['EVM Votes'].astype(int)
table['Postal Votes'] = table['Postal Votes'].astype(int)
table['Total Votes'] = table['Total Votes'].astype(int)
table['% of Votes'] = table['% of Votes'].astype(float)
table['Constituency'] = table['Constituency'].astype(int)
table['Rank']=table.groupby(by='Constituency')['Total Votes'].rank('first', ascending=False)
table.sort_values(by=['Constituency', 'Rank'], inplace=True)
table.reset_index(drop=True, inplace=True)
table

,S.N.,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,Constituency,Rank
0,2,RAJ KARAN KHATRI,Bharatiya Janata Party,86463,752,87215,49.91,1,1.0
1,3,SHARAD KUMAR,Aam Aadmi Party,78395,224,78619,44.99,1,2.0
2,1,ARUNA,Indian National Congress,6724,58,6782,3.88,1,3.0
3,8,NOTA,None of the Above,973,8,981,0.56,1,4.0
4,4,ANIL KUMAR SINGH,Communist Party of India (Marxist-Leninist) ...,328,0,328,0.19,1,5.0
...,...,...,...,...,...,...,...,...,...
764,10,SANJAY MISHRA (SITARAM),Nationalist Congress Party,165,0,165,0.08,70,12.0
765,9,SEEMA,Peoples Party of India (Democratic),128,0,128,0.06,70,13.0
766,6,NEERAJ,New India United Party,100,0,100,0.05,70,14.0
767,8,YOGENDER,Bharatiya Sampuran Krantikari Party,89,1,90,0.04,70,15.0


In [20]:
table.pivot_table(index='Constituency', columns='Abb', values='Total Votes', aggfunc='sum', observed=True).reset_index()

KeyError: 'Abb'

,Total Votes
0,5.0
1,4.0
2,2.0
3,1.0
4,6.0
...,...
880,4.0
881,2.0
882,7.0
883,8.0


In [ ]:
table_summary = table.pivot_table(index='Constituency', columns='Party', values='Total Votes', aggfunc='sum').reset_index()

,S.N.,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,Constituency,Name,Color,Abb
0,1,OM PARKASH GUJJAR,Aam Aadmi Party,592,0,592,0.71,1,Aam Aadmi Party,#0072B0,AAP
1,2,CHARAN SINGH,Bahujan Samaj Party,843,0,843,1.01,1,Bahujan Samaj Party,#22409A,BSP
2,3,PARDEEP CHAUDHARY,Indian National Congress,25979,0,25979,31.2,1,Indian National Congress,#19AAED,INC
3,4,SHAKTI RANI SHARMA,Bharatiya Janata Party,34670,0,34670,41.64,1,Bharatiya Janata Party,#FF9933,BJP
4,5,AMIT SHARMA,Independent,301,0,301,0.36,1,Independent,#DCDCDC,IND
...,...,...,...,...,...,...,...,...,...,...,...
880,9,GIRRAJ SHARMA,Independent,6856,0,6856,9.11,90,Independent,#DCDCDC,IND
881,10,LALIT NAGAR S/O LATE BHARAT SINGH NAGAR,Independent,15138,0,15138,20.12,90,Independent,#DCDCDC,IND
882,11,LALIT NAGAR S/O VIJAY PAL,Independent,340,0,340,0.45,90,Independent,#DCDCDC,IND
883,12,SANDEEP MEHTA,Independent,219,0,219,0.29,90,Independent,#DCDCDC,IND


In [ ]:
parties = pd.read_csv('https://drive.google.com/uc?id=1Sah6rUK2U3W63C1QGos4CMJl81Otx6CX')
parties

,Name,Color,Abb
0,Aam Aadmi Party,#0072B0,AAP
1,AJSU Party,#6827B5,AJSU
2,All India Anna Dravida Munnetra Kazhagam,#009933,AIADMK
3,All India Forward Bloc,#AC1313,AIFB
4,All India Majlis-E-Ittehadul Muslimeen,#0C6B4B,AIMIM
...,...,...,...
87,Voice of the People Party,#0044FF,VPP
88,YSR Congress Party,#1569C7,YSRCP
89,Yuvajana Sramika Rythu Congress Party,#1569C7,YSRCP
90,Zoram Peoples Movement,#FFF700,ZPM


In [ ]:
AAP = table[(table.Party=='Aam Aadmi Party') | (table.Party=='None of the Above')]
AAP = AAP.pivot_table(index='Constituency', columns='Party', values='Total Votes', aggfunc='sum').reset_index()
AAP['>NOTA'] = AAP['Aam Aadmi Party'] > AAP['None of the Above']
AAP

Party,Constituency,Aam Aadmi Party,None of the Above,>NOTA
0,1,592,542,True
1,2,1988,535,False
2,3,2239,242,False
3,4,198,237,False
4,5,842,867,False
...,...,...,...,...
85,86,331,391,False
86,87,712,672,True
87,88,1255,455,False
88,89,365,383,False


Party,Constituency,Aam Aadmi Party,None of the Above
0,1,592,542
1,2,1988,535
2,3,2239,242
3,4,198,237
4,5,842,867
...,...,...,...
85,86,331,391
86,87,712,672
87,88,1255,455
88,89,365,383
